**Check derivative of**  $$\sum\limits_{i=1} ^ {N} Q_{N}^{(i)} \prod_{\substack{j=1 \\ j \neq i}} ^ N Q_{D}^{(i)}$$

In [ ]:
def get_utility_numerator(player, opponents):
    expr = 0
    for opponent in opponents:
        products = [get_Q_D(player, op) for op in opponents if op != opponent]
        expr += get_Q_N(player, opponent) * np.prod(products)
    return expr

In [ ]:
def get_utility_denominator(player, opponents):
    return np.prod([get_Q_D(player, opponent) for opponent in opponents])

**check the utility**

In [ ]:
for seed in range(1000):
    num_players = 5
    np.random.seed(seed)
    opponents = [[np.random.random() for _ in range(4)] for _ in range(num_players)]
    
    np.random.seed(seed + 1000)
    player = [np.random.random() for _ in range(4)]
    
    lhs = get_utility_numerator(player, opponents) / (get_utility_denominator(player, opponents) * num_players)
    rhs = opt_mo.tournament_utility(player, opponents)
    
    assert np.isclose(lhs, rhs)

**check the derivative of the numerator**

In [ ]:
utility_numerator = get_utility_numerator(p, [q, k]).expand()

In [ ]:
derivative_utility_numerator = [sym.diff(utility_numerator, i) for i in p]

In [ ]:
derivative_utility_numerator = [expr.expand() for expr in derivative_utility_numerator]

In [ ]:
def get_lhs_numerator_derivative(player, opponents):
    first_element = 0
    for opponent in opponents:
        products = [get_Q_D(player, op) for op in opponents if op != opponent]
        first_element += get_Q_N_derivative(player, opponent) * np.prod(products)
        
    second_element = 0
    for opponent in opponents:
        temp = [op for op in opponents if op != opponent]
        sums = [get_Q_D_derivative(player, op) * np.prod([get_Q_D(player, o) for o in temp if o != op])
                     for op in temp]
        second_element += get_Q_N(player, opponent) * sum(sums)
    
    return (first_element + second_element)

In [ ]:
written_derivative = get_lhs_numerator_derivative(p, [q, k])

In [ ]:
written_derivative = [der.expand() for der in written_derivative]

In [ ]:
for i in tqdm.tqdm(range(4)):
    assert (written_derivative[i] - derivative_utility_numerator[i]).simplify() == 0

**Check derivative of**  $$ \prod_{\substack{i= 1}} ^ N Q_{D}^{(i)}$$

In [ ]:
def get_denominator_derivative(player, opponents):
    expr = 0
    for opponent in opponents:
        products = [get_Q_D(player, op) for op in opponents if op != opponent]
        expr += get_Q_D_derivative(player, opponent) * np.prod(products)
    
    return expr

In [ ]:
denominator = np.prod([get_Q_D(p, opponent) for opponent in [q, k]])

In [ ]:
derivative_of_denominator = [sym.diff(denominator, i).expand() for i in p]

In [ ]:
written_denominator_derivative = get_denominator_derivative(p, [q, k])

In [ ]:
written_denominator_derivative = [expr.expand() for expr in written_denominator_derivative]

In [ ]:
for i in tqdm.tqdm(range(4)):
    assert (derivative_of_denominator[i] - written_denominator_derivative[i]).simplify() == 0

In [3]:
def get_rhs_numerator_derivative_written(player, opponents):
    
    first_element = get_utility_numerator(player, opponents)
    second_element = get_denominator_derivative(player, opponents)
    
    return first_element * second_element

In [ ]:
for seed in range(100):
    num_players = 5
    np.random.seed(seed)
    opponents = [[np.random.random() for _ in range(4)] for _ in range(num_players)]
    
    np.random.seed(seed + 1000)
    player = [np.random.random() for _ in range(4)]
    
    written_derivative = get_written_derivative(player, opponents) * (1 / num_players)
    
    utility = opt_mo.tournament_utility(p, opponents)
    utility_derivative = [sym.diff(utility, i) for i in p]
    utility_derivative = [expr.subs({p_1: player[0], p_2: player[1], p_3: player[2], p_4: player[3]}) for expr
                          in utility_derivative]
    
    differences = written_derivative - utility_derivative
    for difference in differences:
        assert np.isclose(round(difference, 10), 0)